- [Spark Data Operations](https://github.com/PacktPublishing/Mastering-Big-Data-Analytics-with-PySpark/blob/master/Section%202%20-%20Working%20with%20PySpark/2.5/2.5%20-%20Spark%20Data%20Operations.ipynb)

- [Spark SQL functions](https://docs.google.com/spreadsheets/d/1AuvsTCTxzx1wBMkG-rvfwNpuoQoG9ISuXqQZwj-GGgQ/edit#gid=451439913)

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession.builder.appName("chapter-06-types-functions").getOrCreate()

import os
SPARK_BOOK_DATA_PATH = os.environ['SPARK_BOOK_DATA_PATH']

In [2]:
from IPython.display import display, clear_output

In [3]:
file_path = SPARK_BOOK_DATA_PATH + "/data/retail-data/by-day/2010-12-01.csv"
df = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(file_path)

In [4]:
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
df.show(10,False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |22752

In [6]:
df.count()

3108

In [7]:
spark.sql("select count(*) from dfTable").show()

+--------+
|count(1)|
+--------+
|    3108|
+--------+



#### lit()

In [8]:
df.select(F.lit(5), F.lit("five"), F.lit(5.0)).show(2)

+---+----+---+
|  5|five|5.0|
+---+----+---+
|  5|five|5.0|
|  5|five|5.0|
+---+----+---+
only showing top 2 rows



In [9]:
df.select(F.round(F.lit("2.515"), 2), F.bround(F.lit("2.5"))).show(2)

+---------------+--------------+
|round(2.515, 2)|bround(2.5, 0)|
+---------------+--------------+
|           2.52|           2.0|
|           2.52|           2.0|
+---------------+--------------+
only showing top 2 rows



In [10]:
df.where(F.col("InvoiceNo") != 536365).select("InvoiceNo", "Description").show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [11]:
# complex filter
priceFilter = F.col("UnitPrice") > 600
descripFilter = F.instr(df.Description, "POSTAGE") > 0
df.where(df.StockCode.isin("DOT") & (priceFilter | descripFilter)).show(5)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [12]:
DOTCodeFilter = F.col("StockCode") == "DOT"
priceFilter = F.col("UnitPrice") > 600
descripFilter = F.instr(F.col("Description"), "POSTAGE") >= 1
df2 = (df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))
    .where("isExpensive")
    .select("*")
      )
df2.show(5)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|       true|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|       true|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+



In [13]:
df3 = df2.withColumn("below600", F.expr("UnitPrice < 600")).select("*")

df3.show(5)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+--------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|below600|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+--------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|       true|    true|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|       true|   false|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+--------+



In [14]:
df3.where(F.col("isExpensive") & F.col("below600")).select("*").show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+--------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|isExpensive|below600|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+--------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|       true|    true|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+-----------+--------+



In [15]:
fabricatedQuantity = F.pow(F.col("Quantity") * F.col("UnitPrice"), 2) + 5
(df.select("CustomerId", "Quantity", "UnitPrice", 
           F.round(fabricatedQuantity,2).alias("fakeQuantity"))
    .show(2))

+----------+--------+---------+------------+
|CustomerId|Quantity|UnitPrice|fakeQuantity|
+----------+--------+---------+------------+
|   17850.0|       6|     2.55|      239.09|
|   17850.0|       6|     3.39|      418.72|
+----------+--------+---------+------------+
only showing top 2 rows



In [16]:
df.selectExpr(
    "CustomerId",
    "Quantity", 
    "UnitPrice",
    "round((POWER((Quantity * UnitPrice), 2.0) + 5),3) as fakeQuantity"
).show(2)

+----------+--------+---------+------------+
|CustomerId|Quantity|UnitPrice|fakeQuantity|
+----------+--------+---------+------------+
|   17850.0|       6|     2.55|      239.09|
|   17850.0|       6|     3.39|     418.716|
+----------+--------+---------+------------+
only showing top 2 rows



In [17]:
sql_stmt = """
select 
    CustomerId,
    Quantity, 
    UnitPrice,
    round(POWER((Quantity * UnitPrice), 2.0) + 5,3) as fakeQuantity
from 
    dfTable
"""
spark.sql(sql_stmt).show(2)

+----------+--------+---------+------------+
|CustomerId|Quantity|UnitPrice|fakeQuantity|
+----------+--------+---------+------------+
|   17850.0|       6|     2.55|      239.09|
|   17850.0|       6|     3.39|     418.716|
+----------+--------+---------+------------+
only showing top 2 rows



#### describe()

In [18]:
display(df.describe().toPandas())

,summary,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,count,3108,3108,3098,3108,3108,3108,1968,3108
1,mean,536516.684944841,27834.304044117645,None,8.627413127413128,None,4.151946589446603,15661.388719512195,None
2,stddev,72.89447869788873,17407.897548583845,None,26.371821677029203,None,15.638659854603892,1854.4496996893627,None
3,min,536365,10002,4 PURPLE FLOCK DINNER CANDLES,-24,2010-12-01 08:26:00,0.0,12431.0,Australia
4,max,C536548,POST,ZINC WILLIE WINKIE CANDLE STICK,600,2010-12-01 17:35:00,607.49,18229.0,United Kingdom


#### stat.corr()  and crosstab()

In [19]:
df.stat.corr("Quantity", "UnitPrice")

-0.04112314436835551

In [20]:
df.select(F.corr("Quantity", "UnitPrice").alias("qty_price_corr")).show()

+--------------------+
|      qty_price_corr|
+--------------------+
|-0.04112314436835551|
+--------------------+



In [21]:
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError) # 2.51

[2.51]

In [22]:
display(df.stat.crosstab("StockCode", "Quantity").toPandas())

,StockCode_Quantity,-1,-10,-12,-2,-24,-3,-4,-5,-6,...,60,600,64,7,70,72,8,80,9,96
0,22578,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21327,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22064,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21080,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22219,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1346,47563A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1347,22224,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1348,46000S,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1349,22680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df_1 = df.filter(F.isnull(F.col("StockCode"))).select("*")

In [26]:
df_1.count()

0

In [28]:
df.count()

3108

In [29]:
df_1 = df.filter(F.expr("Quantity < 0")).select("*")
df_1.count()

27

In [32]:
df_1 = df.filter("Quantity > 0").select("*")
df_1.count()

3081

In [33]:
display(df_1.stat.crosstab("StockCode", "Quantity").toPandas())

,StockCode_Quantity,1,10,100,11,12,120,128,13,14,...,60,600,64,7,70,72,8,80,9,96
0,22578,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21327,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,22064,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21080,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22219,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,47563A,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1344,22224,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1345,46000S,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1346,22680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
df.stat.freqItems(["StockCode", "Quantity"]).show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|StockCode_freqItems                                                                                                                                                    

#### monotonically_increasing_id()

In [34]:
df.select(F.monotonically_increasing_id()).show(10)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
+-----------------------------+
only showing top 10 rows



In [35]:
df = df.withColumn("Id", F.monotonically_increasing_id())

display(df.toPandas())

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,4
...,...,...,...,...,...,...,...,...,...
3103,536597,35271S,GOLD PRINT PAPER BAG,14,2010-12-01 17:35:00,0.19,18011.0,United Kingdom,3103
3104,536597,21380,WOODEN HAPPY BIRTHDAY GARLAND,1,2010-12-01 17:35:00,2.95,18011.0,United Kingdom,3104
3105,536597,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,1,2010-12-01 17:35:00,0.85,18011.0,United Kingdom,3105
3106,536597,21221,SET/4 BADGES CUTE CREATURES,5,2010-12-01 17:35:00,1.25,18011.0,United Kingdom,3106


In [54]:
df.select(F.collect_set("Country")).show(truncate=False)

+-----------------------------------------------------------------------+
|collect_set(Country)                                                   |
+-----------------------------------------------------------------------+
|[France, Australia, EIRE, Norway, Germany, United Kingdom, Netherlands]|
+-----------------------------------------------------------------------+



In [36]:
df.count()

3108

In [37]:
df.select(F.max("Id")).show()

+-------+
|max(Id)|
+-------+
|   3107|
+-------+



In [40]:
df.rdd.getNumPartitions()

1

In [41]:
df = df.repartition(3)

In [42]:
df.rdd.getNumPartitions()

3

In [43]:
df = df.withColumn("Id2", F.monotonically_increasing_id())

In [44]:
df.select(F.max("Id"), F.max("Id2")).show()

+-------+-----------+
|max(Id)|   max(Id2)|
+-------+-----------+
|   3107|17179870219|
+-------+-----------+



when df has more than 1 partitions, monotonically_increasing_id() produces gap

In [55]:
df = df.drop("Id3")

#### initcap, lower, upper

In [56]:
df.select(F.initcap(F.col("Description"))).show(5, False)  # False - display column with unlimited width

+----------------------------------+
|initcap(Description)              |
+----------------------------------+
|Modern Floral Stationery Set      |
|Ceramic Strawberry Cake Money Bank|
|Ceramic Strawberry Cake Money Bank|
|Sweetheart Ceramic Trinket Box    |
|Sweetheart Ceramic Trinket Box    |
+----------------------------------+
only showing top 5 rows



In [57]:
(
df.select(F.col("Description"),
    F.initcap(F.col("Description")),
    F.lower(F.col("Description")),
    F.upper(F.col("Description")))
    .show(2, False)
)


+----------------------------------+----------------------------------+----------------------------------+----------------------------------+
|Description                       |initcap(Description)              |lower(Description)                |upper(Description)                |
+----------------------------------+----------------------------------+----------------------------------+----------------------------------+
|MODERN FLORAL STATIONERY SET      |Modern Floral Stationery Set      |modern floral stationery set      |MODERN FLORAL STATIONERY SET      |
|CERAMIC STRAWBERRY CAKE MONEY BANK|Ceramic Strawberry Cake Money Bank|ceramic strawberry cake money bank|CERAMIC STRAWBERRY CAKE MONEY BANK|
+----------------------------------+----------------------------------+----------------------------------+----------------------------------+
only showing top 2 rows



#### ltrim(), rtrim(), trim()

strip spaces (leading, trailing or both)

#### lpad(), rpad()

pad `char` left or right

In [58]:
(df.select(
    F.ltrim(F.lit("    HELLO    ")).alias("ltrim"),
    F.rtrim(F.lit("    HELLO    ")).alias("rtrim"),
    F.trim(F.lit("    HELLO    ")).alias("trim"),
    F.lpad(F.lit("Hello"), 7, " ").alias("lp"),
    F.rpad(F.lit("Hi"), 5, " ").alias("rp"))
    .show(1))

+---------+---------+-----+-------+-----+
|    ltrim|    rtrim| trim|     lp|   rp|
+---------+---------+-----+-------+-----+
|HELLO    |    HELLO|HELLO|  Hello|Hi   |
+---------+---------+-----+-------+-----+
only showing top 1 row



#### translate()
map char to new one

In [59]:
(df.select(F.col("Description"), 
           F.translate(F.col("Description"), "LEET", "1337"))
  .show(2,False))

+----------------------------------+----------------------------------+
|Description                       |translate(Description, LEET, 1337)|
+----------------------------------+----------------------------------+
|MODERN FLORAL STATIONERY SET      |MOD3RN F1ORA1 S7A7ION3RY S37      |
|CERAMIC STRAWBERRY CAKE MONEY BANK|C3RAMIC S7RAWB3RRY CAK3 MON3Y BANK|
+----------------------------------+----------------------------------+
only showing top 2 rows



#### rlike()

#### regexp_replace()

match and replace

#### regexp_extract()

match and extract

In [68]:
regex_string = "BLACK|WHITE|RED|GREEN|BLUE|PINK"

In [69]:
df.filter(F.col("Description").rlike(f"^.*{regex_string}.*$")).select("Description").show(5, truncate=False)

+-----------------------------------+
|Description                        |
+-----------------------------------+
|CHARLOTTE BAG PINK POLKADOT        |
|FRENCH BLUE METAL DOOR SIGN 3      |
|FELTCRAFT HAIRBANDS PINK AND WHITE |
|SET OF 72 GREEN PAPER DOILIES      |
|WOODEN ADVENT CALENDAR RED         |
+-----------------------------------+
only showing top 5 rows



In [70]:
(df
 .filter(F.col("Description").rlike(f"^.*{regex_string}.*$"))
 .select(
    F.col("Description"),
    F.regexp_replace(F.col("Description"), regex_string, "<COLOR>").alias("color_clean"))
  .show(5, False))

+-----------------------------------+----------------------------------------+
|Description                        |color_clean                             |
+-----------------------------------+----------------------------------------+
|CHARLOTTE BAG PINK POLKADOT        |CHARLOTTE BAG <COLOR> POLKADOT          |
|FRENCH BLUE METAL DOOR SIGN 3      |FRENCH <COLOR> METAL DOOR SIGN 3        |
|FELTCRAFT HAIRBANDS PINK AND WHITE |FELTCRAFT HAIRBANDS <COLOR> AND <COLOR> |
|SET OF 72 GREEN PAPER DOILIES      |SET OF 72 <COLOR> PAPER DOILIES         |
|WOODEN ADVENT CALENDAR RED         |WOODEN ADVENT CALENDAR <COLOR>          |
+-----------------------------------+----------------------------------------+
only showing top 5 rows



In [36]:
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
(df.select(
    F.col("Description"),
    F.regexp_extract(F.col("Description"), extract_str, 1).alias("color_clean"))
  .show(5,False))

+-----------------------------------+-----------+
|Description                        |color_clean|
+-----------------------------------+-----------+
|WHITE HANGING HEART T-LIGHT HOLDER |WHITE      |
|WHITE METAL LANTERN                |WHITE      |
|CREAM CUPID HEARTS COAT HANGER     |           |
|KNITTED UNION FLAG HOT WATER BOTTLE|           |
|RED WOOLLY HOTTIE WHITE HEART.     |RED        |
+-----------------------------------+-----------+
only showing top 5 rows



#### instr() 
find a subsring

In [71]:
containsBlack = F.instr(F.col("Description"), "BLACK") > 0
containsWhite = F.instr(F.col("Description"), "WHITE") > 0
(df.withColumn("hasSimpleColor", containsBlack | containsWhite)
   .where("hasSimpleColor")
   .select("Description", "hasSimpleColor")
   .show(5, False))

+-----------------------------------+--------------+
|Description                        |hasSimpleColor|
+-----------------------------------+--------------+
|FELTCRAFT HAIRBANDS PINK AND WHITE |true          |
|WOOD S/3 CABINET ANT WHITE FINISH  |true          |
|WOOD S/3 CABINET ANT WHITE FINISH  |true          |
|EDWARDIAN PARASOL BLACK            |true          |
|WHITE LOVEBIRD LANTERN             |true          |
+-----------------------------------+--------------+
only showing top 5 rows



#### locate() - construct columns dynamically

In [72]:
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
    return F.locate(color_string.upper(), column)\
          .cast("boolean")\
          .alias("is_" + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(F.expr("*")) # has to a be Column type

In [73]:
(df.select(*selectedColumns)
   .where(F.expr("is_white OR is_red"))
   .select("Description","is_white","is_red")
   .show(3, False))

+-----------------------------------+--------+------+
|Description                        |is_white|is_red|
+-----------------------------------+--------+------+
|FELTCRAFT HAIRBANDS PINK AND WHITE |true    |false |
|WOODEN ADVENT CALENDAR RED         |false   |true  |
|WOOD S/3 CABINET ANT WHITE FINISH  |true    |false |
+-----------------------------------+--------+------+
only showing top 3 rows



#### Datetime

- current_date()  
- current_timestamp()

In [74]:
# from pyspark.sql.functions import current_date, current_timestamp

dateDF = (
    spark.range(10)
    .withColumn("today", F.current_date())
    .withColumn("now", F.current_timestamp())
)

dateDF.createOrReplaceTempView("dateTable")

dateDF.show(4, False)

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2021-06-04|2021-06-04 20:25:36.507|
|1  |2021-06-04|2021-06-04 20:25:36.507|
|2  |2021-06-04|2021-06-04 20:25:36.507|
|3  |2021-06-04|2021-06-04 20:25:36.507|
+---+----------+-----------------------+
only showing top 4 rows



#### to_date(), to_timestamp(), date_add(), date_sub(), datediff(), months_between()

see [additional examples](https://github.com/wgong/py4kids/blob/master/lesson-17-pyspark/spark-guide/notebook/chapter-06-udf_datetime.ipynb) using `udf` to parse datetime

In [75]:
spark.sql("select id, today, date_add(today, -3) as past from dateTable limit 3").show()

+---+----------+----------+
| id|     today|      past|
+---+----------+----------+
|  0|2021-06-04|2021-06-01|
|  1|2021-06-04|2021-06-01|
|  2|2021-06-04|2021-06-01|
+---+----------+----------+



In [76]:
(
dateDF
    .select("id", 
            F.date_sub(F.col("today"), 3).alias("past"), 
            "today", 
            F.date_add(F.col("today"), 5).alias("future"))
    .show(3)
)

+---+----------+----------+----------+
| id|      past|     today|    future|
+---+----------+----------+----------+
|  0|2021-06-01|2021-06-04|2021-06-09|
|  1|2021-06-01|2021-06-04|2021-06-09|
|  2|2021-06-01|2021-06-04|2021-06-09|
+---+----------+----------+----------+
only showing top 3 rows



How to work around limitation that 2nd arg of date_add() must be literal `int` value

https://stackoverflow.com/questions/46956026/how-to-convert-column-with-string-type-to-int-form-in-pyspark-data-frame

In [78]:
(
dateDF
    .withColumn("id_days", (F.col("id")+1).cast(IntegerType()))
    .withColumn("past", F.expr("date_sub(today, id_days)"))
    .withColumn("future", F.expr("date_add(today, 2*id_days)"))
    .select("id", 
            "past", 
            "today", 
            "future")
    .show(10)
)

+---+----------+----------+----------+
| id|      past|     today|    future|
+---+----------+----------+----------+
|  0|2021-06-03|2021-06-04|2021-06-06|
|  1|2021-06-02|2021-06-04|2021-06-08|
|  2|2021-06-01|2021-06-04|2021-06-10|
|  3|2021-05-31|2021-06-04|2021-06-12|
|  4|2021-05-30|2021-06-04|2021-06-14|
|  5|2021-05-29|2021-06-04|2021-06-16|
|  6|2021-05-28|2021-06-04|2021-06-18|
|  7|2021-05-27|2021-06-04|2021-06-20|
|  8|2021-05-26|2021-06-04|2021-06-22|
|  9|2021-05-25|2021-06-04|2021-06-24|
+---+----------+----------+----------+



In [79]:
dateDF.withColumn("week_ago", F.date_sub(F.col("today"), 7))\
    .select(F.datediff(F.col("week_ago"), F.col("today")))\
    .show(1)

+-------------------------+
|datediff(week_ago, today)|
+-------------------------+
|                       -7|
+-------------------------+
only showing top 1 row



In [83]:
(dateDF
    .withColumn("start", F.to_date(F.lit("2016-01-01")))
    .withColumn("end", F.to_date(F.lit("2017-05-22")))
    .withColumn("month_diff", F.expr("months_between(start, end)"))
    .select("start", "end", "month_diff")
    .show(1)
)

+----------+----------+------------+
|     start|       end|  month_diff|
+----------+----------+------------+
|2016-01-01|2017-05-22|-16.67741935|
+----------+----------+------------+
only showing top 1 row



reformat date

In [64]:
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
    F.to_date(F.lit("2017-12-11"), dateFormat).alias("date1"),
    F.to_date(F.lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")
cleanDateDF.show()

+----------+----------+
|     date1|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [65]:
spark.sql("select * from dateTable2").show()

+----------+----------+
|     date1|     date2|
+----------+----------+
|2017-11-12|2017-12-20|
+----------+----------+



In [66]:
cleanDateDF.select(F.to_timestamp(F.col("date1"), dateFormat))\
    .show()

+-----------------------------------+
|to_timestamp(`date1`, 'yyyy-dd-MM')|
+-----------------------------------+
|                2017-11-12 00:00:00|
+-----------------------------------+



#### na.drop()

remove rows with missing values

#### na.fill(), na.replace()

replace missing values

In [6]:
display(df.describe().toPandas())

,summary,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,count,3108,3108,3098,3108,3108,3108,1968,3108
1,mean,536516.684944841,27834.304044117645,None,8.627413127413128,None,4.151946589446603,15661.388719512195,None
2,stddev,72.89447869788873,17407.897548583845,None,26.371821677029203,None,15.638659854603892,1854.4496996893627,None
3,min,536365,10002,4 PURPLE FLOCK DINNER CANDLES,-24,2010-12-01 08:26:00,0.0,12431.0,Australia
4,max,C536548,POST,ZINC WILLIE WINKIE CANDLE STICK,600,2010-12-01 17:35:00,607.49,18229.0,United Kingdom


`CustomerID` has many `na` value

In [84]:
df.where("CustomerID is null").select("*").limit(5).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----+---+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|  Id|Id2|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----+---+
|   536544|    22375|AIRLINE BAG VINTA...|       2|2010-12-01 14:32:00|     8.47|      null|United Kingdom|1564|  3|
|   536544|   35095A|BLUE VICTORIAN FA...|       2|2010-12-01 14:32:00|     0.84|      null|United Kingdom|1713|  6|
|   536592|    22174|          PHOTO CUBE|       5|2010-12-01 17:06:00|     3.36|      null|United Kingdom|2666|  8|
|   536544|    22335|HEART DECORATION ...|       4|2010-12-01 14:32:00|     1.28|      null|United Kingdom|1552|  9|
|   536592|    85227|SET OF 6 3D KIT C...|       2|2010-12-01 17:06:00|     1.66|      null|United Kingdom|3009| 10|
+---------+---------+--------------------+--------+-------------

In [85]:
df_na = df.where("CustomerID is null")

In [86]:
df_na.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----+---+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|  Id|Id2|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----+---+
|   536544|    22375|AIRLINE BAG VINTA...|       2|2010-12-01 14:32:00|     8.47|      null|United Kingdom|1564|  3|
|   536544|   35095A|BLUE VICTORIAN FA...|       2|2010-12-01 14:32:00|     0.84|      null|United Kingdom|1713|  6|
|   536592|    22174|          PHOTO CUBE|       5|2010-12-01 17:06:00|     3.36|      null|United Kingdom|2666|  8|
|   536544|    22335|HEART DECORATION ...|       4|2010-12-01 14:32:00|     1.28|      null|United Kingdom|1552|  9|
|   536592|    85227|SET OF 6 3D KIT C...|       2|2010-12-01 17:06:00|     1.66|      null|United Kingdom|3009| 10|
+---------+---------+--------------------+--------+-------------

In [87]:
df_na.count()

1140

In [88]:
df_na.where("Description is null").count()

10

In [89]:
df_na.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- Id: long (nullable = false)
 |-- Id2: long (nullable = false)



In [90]:
fill_cols_vals = {"CustomerID": -9999, "Description" : "MISSING_VALUE"}
df_na = df_na.na.fill(fill_cols_vals)

In [91]:
df_na.where("Description is null").count()

0

In [92]:
df_na.where("CustomerID is null").count()

0

In [93]:
df_na.na.fill("all", subset=["StockCode", "InvoiceNo"])

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string, Id: bigint, Id2: bigint]

In [94]:
df_na.filter(F.col("Description") == "MISSING_VALUE").show(5,False)

+---------+---------+-------------+--------+-------------------+---------+----------+--------------+----+----------+
|InvoiceNo|StockCode|Description  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |Id  |Id2       |
+---------+---------+-------------+--------+-------------------+---------+----------+--------------+----+----------+
|536554   |84670    |MISSING_VALUE|23      |2010-12-01 14:35:00|0.0      |-9999.0   |United Kingdom|2026|132       |
|536547   |37509    |MISSING_VALUE|1       |2010-12-01 14:33:00|0.0      |-9999.0   |United Kingdom|1972|166       |
|536550   |85044    |MISSING_VALUE|1       |2010-12-01 14:34:00|0.0      |-9999.0   |United Kingdom|1988|232       |
|536549   |85226A   |MISSING_VALUE|1       |2010-12-01 14:34:00|0.0      |-9999.0   |United Kingdom|1987|345       |
|536589   |21777    |MISSING_VALUE|-10     |2010-12-01 16:50:00|0.0      |-9999.0   |United Kingdom|2406|8589934739|
+---------+---------+-------------+--------+-------------------+

In [95]:
df_na = df_na.na.replace([-9999.0], [-11111], "CustomerID")

In [96]:
df_na.filter(F.col("CustomerID") == -11111).show(5,False)

+---------+---------+---------------------------------+--------+-------------------+---------+----------+--------------+----+---+
|InvoiceNo|StockCode|Description                      |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |Id  |Id2|
+---------+---------+---------------------------------+--------+-------------------+---------+----------+--------------+----+---+
|536544   |22375    |AIRLINE BAG VINTAGE JET SET BROWN|2       |2010-12-01 14:32:00|8.47     |-11111.0  |United Kingdom|1564|3  |
|536544   |35095A   |BLUE VICTORIAN FABRIC OVAL BOX   |2       |2010-12-01 14:32:00|0.84     |-11111.0  |United Kingdom|1713|6  |
|536592   |22174    |PHOTO CUBE                       |5       |2010-12-01 17:06:00|3.36     |-11111.0  |United Kingdom|2666|8  |
|536544   |22335    |HEART DECORATION PAINTED ZINC    |4       |2010-12-01 14:32:00|1.28     |-11111.0  |United Kingdom|1552|9  |
|536592   |85227    |SET OF 6 3D KIT CARDS FOR KIDS   |2       |2010-12-01 17:06:00|1.66  

### Complex type


#### struct()

combine multiple columns into array

In [97]:
complexDF = df.select(F.struct("InvoiceNo", "Description").alias("complex"))

complexDF.createOrReplaceTempView("complexDF")

In [98]:
spark.sql("select * from complexDF").show(5, False)

+----------------------------------------+
|complex                                 |
+----------------------------------------+
|[536551, PLASTERS IN TIN STRONGMAN]     |
|[536544, CREAM SWEETHEART LETTER RACK]  |
|[536538, DOORMAT SPOTTY HOME SWEET HOME]|
|[536592, LIPSTICK PEN BABY PINK]        |
|[536544, RED BABY BUNTING ]             |
+----------------------------------------+
only showing top 5 rows



#### split

convert one column into array type

In [99]:
df.select("Description", F.split(F.col("Description"), " ").alias("desc_words")).show(2, False)

+----------------------------------+----------------------------------------+
|Description                       |desc_words                              |
+----------------------------------+----------------------------------------+
|MODERN FLORAL STATIONERY SET      |[MODERN, FLORAL, STATIONERY, SET]       |
|CERAMIC STRAWBERRY CAKE MONEY BANK|[CERAMIC, STRAWBERRY, CAKE, MONEY, BANK]|
+----------------------------------+----------------------------------------+
only showing top 2 rows



In [100]:
df_tmp = (df.withColumn("array_col", F.split(F.col("Description"), " "))
    .selectExpr("Description", "array_col", "array_col[0]","array_col[1]"))
display(df_tmp.toPandas())

,Description,array_col,array_col[0],array_col[1]
0,MODERN FLORAL STATIONERY SET,"[MODERN, FLORAL, STATIONERY, SET]",MODERN,FLORAL
1,CERAMIC STRAWBERRY CAKE MONEY BANK,"[CERAMIC, STRAWBERRY, CAKE, MONEY, BANK]",CERAMIC,STRAWBERRY
2,CERAMIC STRAWBERRY CAKE MONEY BANK,"[CERAMIC, STRAWBERRY, CAKE, MONEY, BANK]",CERAMIC,STRAWBERRY
3,SWEETHEART CERAMIC TRINKET BOX,"[SWEETHEART, CERAMIC, TRINKET, BOX]",SWEETHEART,CERAMIC
4,SWEETHEART CERAMIC TRINKET BOX,"[SWEETHEART, CERAMIC, TRINKET, BOX]",SWEETHEART,CERAMIC
...,...,...,...,...
3103,PAPER CHAIN KIT VINTAGE CHRISTMAS,"[PAPER, CHAIN, KIT, VINTAGE, CHRISTMAS]",PAPER,CHAIN
3104,PAPER CHAIN KIT VINTAGE CHRISTMAS,"[PAPER, CHAIN, KIT, VINTAGE, CHRISTMAS]",PAPER,CHAIN
3105,PAPER CHAIN KIT VINTAGE CHRISTMAS,"[PAPER, CHAIN, KIT, VINTAGE, CHRISTMAS]",PAPER,CHAIN
3106,PACK OF 6 BIRDY GIFT TAGS,"[PACK, OF, 6, BIRDY, GIFT, TAGS]",PACK,OF


#### size()

array size

In [101]:
df.select("Description",
    F.size(F.split(F.col("Description"), " ")).alias("arr_size"))\
    .show(2, False) # shows 5 and 3

+----------------------------------+--------+
|Description                       |arr_size|
+----------------------------------+--------+
|MODERN FLORAL STATIONERY SET      |4       |
|CERAMIC STRAWBERRY CAKE MONEY BANK|5       |
+----------------------------------+--------+
only showing top 2 rows



#### array_contains

find an element in an array

In [102]:
df.select("Description",
        F.array_contains(F.split(F.col("Description"), " "), "WHITE").alias("has_white")
    ).show(2,False)

+----------------------------------+---------+
|Description                       |has_white|
+----------------------------------+---------+
|MODERN FLORAL STATIONERY SET      |false    |
|CERAMIC STRAWBERRY CAKE MONEY BANK|false    |
+----------------------------------+---------+
only showing top 2 rows



#### explode

denorm array column

In [103]:
df.withColumn("splitted", F.split(F.col("Description"), " "))\
  .withColumn("exploded", F.explode(F.col("splitted")))\
  .select("Description", "InvoiceNo", "exploded")\
  .show(10, False)

+------------------------------+---------+----------+
|Description                   |InvoiceNo|exploded  |
+------------------------------+---------+----------+
|PLASTERS IN TIN STRONGMAN     |536551   |PLASTERS  |
|PLASTERS IN TIN STRONGMAN     |536551   |IN        |
|PLASTERS IN TIN STRONGMAN     |536551   |TIN       |
|PLASTERS IN TIN STRONGMAN     |536551   |STRONGMAN |
|CREAM SWEETHEART LETTER RACK  |536544   |CREAM     |
|CREAM SWEETHEART LETTER RACK  |536544   |SWEETHEART|
|CREAM SWEETHEART LETTER RACK  |536544   |LETTER    |
|CREAM SWEETHEART LETTER RACK  |536544   |RACK      |
|DOORMAT SPOTTY HOME SWEET HOME|536538   |DOORMAT   |
|DOORMAT SPOTTY HOME SWEET HOME|536538   |SPOTTY    |
+------------------------------+---------+----------+
only showing top 10 rows



#### map

create a hash map between 2 columns

In [104]:
df.select("Description", "InvoiceNo", F.create_map(F.col("Description"), F.col("InvoiceNo")).alias("complex_map"))\
  .show(5, False)

+------------------------------+---------+------------------------------------------+
|Description                   |InvoiceNo|complex_map                               |
+------------------------------+---------+------------------------------------------+
|PLASTERS IN TIN STRONGMAN     |536551   |[PLASTERS IN TIN STRONGMAN -> 536551]     |
|CREAM SWEETHEART LETTER RACK  |536544   |[CREAM SWEETHEART LETTER RACK -> 536544]  |
|DOORMAT SPOTTY HOME SWEET HOME|536538   |[DOORMAT SPOTTY HOME SWEET HOME -> 536538]|
|LIPSTICK PEN BABY PINK        |536592   |[LIPSTICK PEN BABY PINK -> 536592]        |
|RED BABY BUNTING              |536544   |[RED BABY BUNTING  -> 536544]             |
+------------------------------+---------+------------------------------------------+
only showing top 5 rows



In [105]:
df.select(F.create_map(F.col("Description"), F.col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("complex_map['WHITE METAL LANTERN']")\
    .show(2)

+--------------------------------+
|complex_map[WHITE METAL LANTERN]|
+--------------------------------+
|                            null|
|                            null|
+--------------------------------+
only showing top 2 rows



In [106]:
df.withColumn("complex_map", F.create_map(F.col("Description"), F.col("InvoiceNo")))\
    .selectExpr("Description", "InvoiceNo", "explode(complex_map)")\
    .show(2, False)

+----------------------------+---------+----------------------------+------+
|Description                 |InvoiceNo|key                         |value |
+----------------------------+---------+----------------------------+------+
|PLASTERS IN TIN STRONGMAN   |536551   |PLASTERS IN TIN STRONGMAN   |536551|
|CREAM SWEETHEART LETTER RACK|536544   |CREAM SWEETHEART LETTER RACK|536544|
+----------------------------+---------+----------------------------+------+
only showing top 2 rows



### Json

In [52]:
jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")


In [53]:
jsonDF.show(2, False)

+-------------------------------------------+
|jsonString                                 |
+-------------------------------------------+
|{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}|
+-------------------------------------------+



In [33]:
jsonDF.select(
    F.get_json_object(F.col("jsonString"), "$.myJSONKey.myJSONValue[1]").alias("column"),
    F.json_tuple(F.col("jsonString"), "myJSONKey")
    ).show(2, False)

+------+-----------------------+
|column|c0                     |
+------+-----------------------+
|2     |{"myJSONValue":[1,2,3]}|
+------+-----------------------+



### pack columns into json

In [34]:
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(F.to_json(F.col("myStruct")))\
  .show(3, False)

+-------------------------------------------------------------------------+
|structstojson(myStruct)                                                  |
+-------------------------------------------------------------------------+
|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
|{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}    |
+-------------------------------------------------------------------------+
only showing top 3 rows



In [35]:
parseSchema = StructType((
  StructField("InvoiceNo",StringType(),True),
  StructField("Description",StringType(),True)))

df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(F.to_json(F.col("myStruct")).alias("newJSON"))\
  .select(F.from_json(F.col("newJSON"), parseSchema).alias("old_json"), F.col("newJSON"))\
    .show(2, False)

+--------------------------------------------+-------------------------------------------------------------------------+
|old_json                                    |newJSON                                                                  |
+--------------------------------------------+-------------------------------------------------------------------------+
|[536365, WHITE HANGING HEART T-LIGHT HOLDER]|{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"}|
|[536365, WHITE METAL LANTERN]               |{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}               |
+--------------------------------------------+-------------------------------------------------------------------------+
only showing top 2 rows



### udf()

user-defined function

In [108]:
udfExampleDF = spark.range(5).toDF("num")

In [109]:
def power3(double_value):
  return float(double_value ** 3)
power3(2.0)

8.0

In [119]:
power3udf = F.udf(power3, FloatType())
# this udf is available to dataframe, not to SQL

In [120]:
spark.sql("show user functions like 'power*'").show()

+--------+
|function|
+--------+
|power3py|
+--------+



In [114]:
udfExampleDF\
    .select("num", power3udf(F.col("num")).alias("num_cubed"))\
    .show(6)

+---+---------+
|num|num_cubed|
+---+---------+
|  0|      0.0|
|  1|      1.0|
|  2|      8.0|
|  3|     27.0|
|  4|     64.0|
+---+---------+



register `udf` to be available to SQL query

In [115]:
spark.udf.register("power3py", power3, DoubleType())
# this udf is available to SQL

<function __main__.power3(double_value)>

In [116]:
spark.sql("show user functions like 'power*'").show()

+--------+
|function|
+--------+
|power3py|
+--------+



In [117]:
spark.sql("select 10, power3py(100)").show()

+---+-------------+
| 10|power3py(100)|
+---+-------------+
| 10|    1000000.0|
+---+-------------+



In [118]:
udfExampleDF.selectExpr("power3py(num)").show(5)

+-------------+
|power3py(num)|
+-------------+
|          0.0|
|          1.0|
|          8.0|
|         27.0|
|         64.0|
+-------------+

